In [1]:
import pandas as pd

In [2]:
# Load the dataset
df = pd.read_csv('transaction_data.csv')

In [3]:
# Convert DATE column to datetime format
df["DATE"] = pd.to_datetime(df["DATE"], origin="1899-12-30", unit="D")

In [4]:
# Define snapshot date (1 day after the last transaction)
snapshot_date = df["DATE"].max() + pd.Timedelta(days=1)

In [5]:

#Calculate RFM metrics
rfm_df = df.groupby("LYLTY_CARD_NBR").agg(
    Recency=("DATE", lambda x: (snapshot_date - x.max()).days),
    Frequency=("TXN_ID", "nunique"),
    Monetary=("TOT_SALES", "sum")
).reset_index()

In [6]:
# Assign RFM scores (1 to 5) using quantiles
rfm_df["R_Score"] = pd.qcut(rfm_df["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm_df["F_Score"] = pd.qcut(rfm_df["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm_df["M_Score"] = pd.qcut(rfm_df["Monetary"], 5, labels=[1, 2, 3, 4, 5])

In [7]:
# Combine RFM scores into a single RFM Score
rfm_df["RFM_Score"] = rfm_df["R_Score"].astype(str) + rfm_df["F_Score"].astype(str) + rfm_df["M_Score"].astype(str)

In [8]:
# Define customer segmentation function
def segment_customer(rfm):
    if rfm["RFM_Score"] in ["555", "554", "544", "545", "543"]:
        return "Best Customers"
    elif rfm["RFM_Score"] in ["444", "445", "433", "432"]:
        return "Loyal Customers"
    elif rfm["RFM_Score"] in ["311", "211", "111"]:
        return "Lost Customers"
    elif rfm["RFM_Score"] in ["511", "522", "521"]:
        return "New Customers"
    elif rfm["RFM_Score"] in ["333", "322", "221"]:
        return "Promising Customers"
    else:
        return "moderate"

In [9]:
# Apply segmentation
rfm_df["Segment"] = rfm_df.apply(segment_customer, axis=1)

In [10]:
# Display the final RFM table
print(rfm_df.head())

   LYLTY_CARD_NBR  Recency  Frequency  Monetary R_Score F_Score M_Score  \
0            1000      257          1       6.0       1       1       1   
1            1002      288          1       2.7       1       1       1   
2            1003      115          2       6.6       2       2       1   
3            1004      241          1       1.9       1       1       1   
4            1005      185          1       2.8       2       1       1   

  RFM_Score              Segment  
0       111       Lost Customers  
1       111       Lost Customers  
2       221  Promising Customers  
3       111       Lost Customers  
4       211       Lost Customers  


In [11]:
rfm_df

,LYLTY_CARD_NBR,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,1000,257,1,6.0,1,1,1,111,Lost Customers
1,1002,288,1,2.7,1,1,1,111,Lost Customers
2,1003,115,2,6.6,2,2,1,221,Promising Customers
3,1004,241,1,1.9,1,1,1,111,Lost Customers
4,1005,185,1,2.8,2,1,1,211,Lost Customers
...,...,...,...,...,...,...,...,...,...
72632,2370651,332,1,13.0,1,2,2,122,moderate
72633,2370701,205,1,7.2,1,2,1,121,moderate
72634,2370751,273,1,9.2,1,2,2,122,moderate
72635,2370961,247,2,18.6,1,3,3,133,moderate


In [12]:
rfm_df['Segment'].value_counts()

Segment
moderate               42958
Best Customers          9566
Lost Customers          8537
Loyal Customers         5520
Promising Customers     4317
New Customers           1739
Name: count, dtype: int64